## Fase 0: Creación de la Arquitectura del Catálogo

**Objetivo:** Construir el esqueleto de nuestro Lakehouse en Unity Catalog.

Inspirados en las mejores prácticas, no trabajaremos en esquemas sueltos. Crearemos una estructura autocontenida y profesional:
1.  **Un Catálogo dedicado**: `sesion_5`. Será el contenedor principal de todo nuestro proyecto.
2.  **Tres Esquemas (Bases de Datos)**: `bronze`, `silver` y `gold`, uno para cada capa de nuestra arquitectura Medallion.
3.  **Tres Volúmenes**: `raw_files`, `silver_tables` y `gold_tables`, para almacenar físicamente los datos de cada capa.

Este paso se ejecuta una sola vez para configurar todo el entorno.

## La Arquitectura Medallion: Un Enfoque Lógico para el Lakehouse

La **Arquitectura Medallion** es un patrón de diseño de datos que organiza lógicamente los datos dentro de un Data Lakehouse en tres capas distintas: **Bronce (raw)**, **Plata (validated)** y **Oro (enriched)**. El objetivo principal es mejorar de manera incremental la calidad, estructura y fiabilidad de los datos a medida que fluyen a través de estas capas. 

Imagina el proceso como el de una refinería: el petróleo crudo (Bronce) se refina para obtener productos más útiles y limpios como la gasolina (Plata), y finalmente se convierte en productos de alto valor y especializados (Oro).

***
### ¿Por Qué Usar la Arquitectura Medallion? La Justificación

Adoptar este enfoque no es solo una cuestión de organización, sino una estrategia que resuelve problemas fundamentales en la gestión de datos a gran escala:

* **Integridad y Confianza en los Datos**: Garantiza que los usuarios de negocio consuman datos de alta calidad (capa Oro), mientras que los procesos de ETL/ELT se construyen sobre una base validada y limpia (capa Plata). Esto evita el problema del "pantano de datos" (Data Swamp), donde los datos crudos y sin gobierno generan desconfianza.
* **Linaje de Datos y Gobernanza Sencillos**: La estructura en capas hace que sea muy fácil rastrear el origen de cualquier dato (linaje) y aplicar controles de seguridad y gobernanza. Por ejemplo, se puede restringir el acceso a los datos crudos (Bronce) y solo permitir a la mayoría de los usuarios acceder a las capas Plata y Oro.
* **Eficiencia en el Procesamiento**: Al tener una capa Plata limpia y modelada, se evita tener que reprocesar y limpiar los datos crudos de la capa Bronce una y otra vez para cada nuevo caso de uso. La capa Plata actúa como una "fuente única de la verdad" optimizada para futuras transformaciones.
* **Democratización del Acceso a los Datos**: Cada capa sirve a un propósito y a una audiencia diferente, permitiendo que tanto los ingenieros de datos, como los científicos de datos y los analistas de negocio trabajen de manera eficiente sin interferir entre sí.

***
### Las Capas en Detalle

#### 🥉 **Capa Bronce (Raw Data)**

Esta es la primera parada de los datos en el Lakehouse.

* **Propósito**: Ingestar y almacenar datos de los sistemas de origen en su **formato nativo y sin procesar**. Es una copia fiel del origen.
* **Características**:
    * **Inmutable y de solo adición (append-only)**: Se guardan todos los datos históricos, sin eliminar ni modificar nada. Si llega una actualización de una fila, se añade como una nueva fila, preservando el historial completo.
    * **Estructura del Origen**: La estructura de las tablas en esta capa debe reflejar la del sistema de origen para facilitar la conciliación.
    * **Base para la Reconstrucción**: Es la capa de respaldo. Si hay errores en las capas Plata u Oro, siempre se pueden reconstruir a partir de los datos crudos y confiables de la capa Bronce.
* **Usuarios Típicos**: Principalmente ingenieros de datos que construyen los pipelines hacia la capa Plata.

#### 🥈 **Capa Plata (Cleansed & Conformed Data)**

Esta capa es la "fuente única de la verdad" del Lakehouse.

* **Propósito**: Proporcionar una visión limpia, validada y modelada de los datos empresariales clave. Aquí es donde los datos crudos se convierten en información de valor.
* **Características**:
    * **Limpieza de Datos**: Se manejan valores nulos, se corrigen tipos de datos y se eliminan duplicados.
    * **Enriquecimiento y Conformación**: Se unen datos de diferentes fuentes para crear una visión más completa. Por ejemplo, se une un pedido con la información del cliente.
    * **Modelado de Datos**: Es aquí donde comúnmente se aplica un **Esquema en Estrella**, creando tablas de hechos y dimensiones que son intuitivas y optimizadas para el análisis.
* **Usuarios Típicos**: Ingenieros de datos, científicos de datos (para exploración y feature engineering) y analistas de negocio avanzados.

#### 🥇 **Capa Oro (Curated Business-Level Data)**

Esta es la capa final, optimizada para el consumo directo por parte del negocio.

* **Propósito**: Entregar datos altamente refinados y agregados, listos para responder a preguntas de negocio específicas.
* **Características**:
    * **Agregaciones de Negocio**: Las tablas en esta capa contienen métricas pre-calculadas (ej. `ventas_totales_por_mes`, `clientes_activos_por_region`).
    * **Optimización para el Rendimiento**: Los datos suelen estar desnormalizados y organizados por proyecto o área de negocio para que las consultas de BI y los modelos de ML sean extremadamente rápidos.
    * **Enfocado en el Caso de Uso**: Mientras que la capa Plata está modelada para toda la empresa, la capa Oro se centra en las necesidades de un departamento o proyecto específico (Finanzas, Marketing, etc.).
* **Usuarios Típicos**: Analistas de negocio, científicos de datos (para entrenamiento de modelos) y cualquier usuario final que consuma dashboards o reportes.

In [0]:
%sql
-- 1. Creamos nuestro propio catálogo para el proyecto
CREATE CATALOG IF NOT EXISTS sesion_5;

-- 2. Creamos los esquemas para cada capa dentro de nuestro nuevo catálogo
CREATE SCHEMA IF NOT EXISTS sesion_5.bronze COMMENT 'Esquema para datos crudos e inmutables';
CREATE SCHEMA IF NOT EXISTS sesion_5.silver COMMENT 'Esquema para datos validados, limpiados y modelados';
CREATE SCHEMA IF NOT EXISTS sesion_5.gold COMMENT 'Esquema para datos agregados y listos para el negocio';

-- 3. Creamos los volúmenes para el almacenamiento físico de los datos
CREATE VOLUME IF NOT EXISTS sesion_5.bronze.raw_files;
CREATE VOLUME IF NOT EXISTS sesion_5.silver.tables;
CREATE VOLUME IF NOT EXISTS sesion_5.gold.tables;

SELECT "Catálogo, esquemas y volúmenes creados exitosamente." as status;

### ✋ Acción Manual: Subir los Archivos a la Capa Bronze

Ahora que la estructura está creada, debes subir los 9 archivos CSV del dataset de Olist a nuestro nuevo volumen.

**Pasos:**
1.  En el menú de la izquierda, ve a **Catalog**.
2.  Navega hasta tu nuevo catálogo: `sesion_5`.
3.  Entra al esquema `bronze`.
4.  Haz clic en el volumen `raw_files`.
5.  Usa el botón **"Upload to this volume"** para subir tus archivos.

**Una vez que los archivos estén subidos, puedes continuar ejecutando el resto del cuaderno.**

## Fase 1: Configuración y Carga de Datos en DataFrames

**Objetivo:** Definir las rutas a nuestra nueva arquitectura y cargar los datos del volumen `bronze` en DataFrames de Spark para su procesamiento.

In [0]:
# 1. Definir la configuración usando nuestra nueva estructura de catálogo
catalog_name = "sesion_5"
bronze_schema = "bronze"
silver_schema = "silver"

# 2. Definir la ruta al volumen de la capa Bronze
bronze_volume_path = f"/Volumes/{catalog_name}/{bronze_schema}/raw_files"

# 3. Cargar todos los datasets desde el volumen Bronze en DataFrames
print(f"Leyendo archivos desde: {bronze_volume_path}...")
options = {"header": "true", "inferSchema": "true"}

customers_df = spark.read.options(**options).csv(f"{bronze_volume_path}/olist_customers_dataset.csv")
order_items_df = spark.read.options(**options).csv(f"{bronze_volume_path}/olist_order_items_dataset.csv")
order_payments_df = spark.read.options(**options).csv(f"{bronze_volume_path}/olist_order_payments_dataset.csv")
orders_df = spark.read.options(**options).csv(f"{bronze_volume_path}/olist_orders_dataset.csv")
products_df = spark.read.options(**options).csv(f"{bronze_volume_path}/olist_products_dataset.csv")
sellers_df = spark.read.options(**options).csv(f"{bronze_volume_path}/olist_sellers_dataset.csv")
category_translation_df = spark.read.options(**options).csv(f"{bronze_volume_path}/product_category_name_translation.csv")

print("\n¡Carga completada! Los DataFrames de la capa Bronze están en memoria.")

## Fase 2: Exploración de Datos (EDA)

**Objetivo:** Entender la estructura, el contenido y la escala de nuestros datos crudos.

Antes de transformar, debemos entender con qué estamos trabajando. Un buen EDA nos permite:
1.  **Validar los Tipos de Datos**: ¿Spark infirió correctamente las fechas y los números?
2.  **Identificar Nulos**: ¿Hay columnas con datos faltantes que debamos tratar?
3.  **Tener una Noción del Volumen**: ¿Estamos hablando de miles o millones de registros?

Realizaremos una exploración básica sobre los DataFrames que acabamos de cargar.

In [0]:
# 1. Revisar los esquemas para validar los tipos de datos
print("--- Esquema de 'orders_df' (Pedidos) ---")
orders_df.printSchema()

# 2. Contar el número de filas para entender la escala
orders_count = orders_df.count()
print(f"\nNúmero de registros en 'orders_df': {orders_count:,}")

# 3. Visualizar una muestra de los datos para entender el contenido
print("\n--- Muestra de datos de 'orders_df' ---")
orders_df.show(5, truncate=False)

## Fase 3: Reflexión - El Plan del Arquitecto 

**Objetivo:** Diseñar nuestro modelo Silver antes de construirlo.

Hemos confirmado que tenemos 9 DataFrames con datos relacionados pero desorganizados. Si un analista quisiera saber "las ventas totales por ciudad", tendría que unir `orders`, `order_items`, `order_payments` y `customers` cada vez que haga la consulta. Esto es ineficiente y propenso a errores.

Aquí es donde aplicamos el **pensamiento arquitectónico**. Nuestro plan es transformar estos datos crudos en un **Esquema en Estrella** robusto y optimizado para el análisis.

* **La Estrella Central (Hechos)**: El proceso de negocio fundamental es el **pedido**. Por lo tanto, crearemos una única tabla de hechos, `fact_pedidos`, que contendrá las métricas (`precio`, `valor_pago`) y las claves para conectarse con su contexto.
* **Los Puntos de la Estrella (Dimensiones)**: El contexto que describe cada hecho son las entidades de negocio:
    * `dim_clientes`: ¿**Quién** compró?
    * `dim_productos`: ¿**Qué** se compró?
    * `dim_vendedores`: ¿**Quién** lo vendió?

Este modelo es el objetivo de nuestra capa **Silver**. En la siguiente fase, lo haremos realidad.

## Fase 4: Creación de Tablas Silver y Comparativa de Métodos

**Objetivo:** Crear nuestras tablas en la capa Silver y entender las dos formas de hacerlo: **PySpark vs. SQL**.

### 4.1. PySpark vs. SQL: Dos Caminos para el Mismo Destino
* **PySpark (Programático)** 🐍: Flexible y potente, ideal para lógica compleja y pipelines automatizados. Manipulamos DataFrames con código Python.
* **Spark SQL (Declarativo)** 📊: Legible y universal. Perfecto para transformaciones de BI. Le *declaramos* el resultado que queremos a la base de datos.

Crearemos la `dim_clientes` con ambos métodos para ver la diferencia.

#### Ejemplo 1: Creando `dim_clientes` con PySpark

In [0]:
%sql
-- Paso 1 (PySpark): Crear una tabla vacía con el esquema correcto
-- Usamos SQL para definir la estructura porque es muy claro y explícito.
CREATE OR REPLACE TABLE sesion_5.silver.dim_clientes_pyspark (
  customer_id STRING NOT NULL,
  customer_unique_id STRING,
  customer_zip_code_prefix STRING,
  customer_city STRING,
  customer_state STRING
)
USING DELTA
COMMENT 'Dimensión de Clientes, creada con PySpark.';

SELECT "Tabla 'dim_clientes_pyspark' creada con el esquema correcto." as status;

In [0]:
from pyspark.sql.functions import col

# Paso 2 (PySpark): Transformar los datos en un DataFrame
dim_clientes_pyspark_df = customers_df.select(
    col("customer_id"),
    col("customer_unique_id"),
    col("customer_zip_code_prefix"),
    col("customer_city"),
    col("customer_state")
).distinct()

# Paso 3 (PySpark): Insertar los datos del DataFrame en la tabla que ya creamos.
# Usamos .insertInto() para añadir los datos a la tabla existente.
# El modo "overwrite" vaciará la tabla antes de insertar, asegurando que no haya datos viejos.
dim_clientes_pyspark_df.write.mode("overwrite").insertInto("sesion_5.silver.dim_clientes_pyspark")

print("Datos insertados correctamente en 'dim_clientes_pyspark'.")
display(spark.table("sesion_5.silver.dim_clientes_pyspark"))

#### Ejemplo 2: Creando `dim_clientes` con Spark SQL

In [0]:
# Paso 1: Registrar el DataFrame como una vista temporal para poder usar SQL sobre él.
customers_df.createOrReplaceTempView("customers_bronze_vw")

print("Vista temporal 'customers_bronze_vw' creada.")

In [0]:
%sql
-- Paso 2: Crear la tabla con el esquema explícito y luego insertar los datos.
-- Este enfoque de dos pasos nos da control total sobre la estructura de la tabla.

-- Primero, creamos la tabla con la columna de la llave primaria definida como NOT NULL.
CREATE OR REPLACE TABLE sesion_5.silver.dim_clientes_sql (
  customer_id STRING NOT NULL,
  customer_unique_id STRING,
  customer_zip_code_prefix STRING,
  customer_city STRING,
  customer_state STRING
)
USING DELTA
COMMENT 'Dimensión de Clientes, creada con SQL y con esquema definido.';

-- Segundo, insertamos los datos en la tabla ya creada.
INSERT INTO sesion_5.silver.dim_clientes_sql
SELECT DISTINCT
  customer_id,
  customer_unique_id,
  customer_zip_code_prefix,
  customer_city,
  customer_state
FROM customers_bronze_vw;

-- Verificamos el resultado
SELECT * FROM sesion_5.silver.dim_clientes_sql LIMIT 5;

Ahora construiremos las tablas `dim_productos`, `dim_vendedores` y, finalmente, la tabla central `fact_pedidos` que une todo el modelo. Seguiremos el mismo patrón de crear vistas temporales sobre los datos Bronze y luego usar SQL para transformar y crear las tablas Silver.

### 5.1. Dimensión de Productos (`dim_productos`)

In [0]:
# 1. Registrar los DataFrames necesarios como vistas temporales
products_df.createOrReplaceTempView("products_bronze_vw")
category_translation_df.createOrReplaceTempView("category_translation_bronze_vw")

print("Vistas temporales 'products_bronze_vw' y 'category_translation_bronze_vw' creadas.")

In [0]:
%sql
-- Crear la tabla de dimensión de productos
-- **CORRECCIÓN**: Añadimos la definición de la columna PK como NOT NULL
CREATE OR REPLACE TABLE sesion_5.silver.dim_productos (
  product_id STRING NOT NULL,
  product_category STRING,
  product_photos_qty INT,
  product_weight_g DOUBLE,
  product_length_cm DOUBLE,
  product_height_cm DOUBLE,
  product_width_cm DOUBLE
)
USING DELTA
COMMENT 'Dimensión de Productos, enriquecida.';

-- Insertamos los datos desde nuestra vista temporal
INSERT INTO sesion_5.silver.dim_productos
SELECT DISTINCT
  p.product_id,
  t.product_category_name_english AS product_category,
  p.product_photos_qty,
  p.product_weight_g,
  p.product_length_cm,
  p.product_height_cm,
  p.product_width_cm
FROM products_bronze_vw p
LEFT JOIN category_translation_bronze_vw t ON p.product_category_name = t.product_category_name;

-- Verificamos
SELECT * FROM sesion_5.silver.dim_productos LIMIT 5;

### 5.2. Dimensión de Vendedores (`dim_vendedores`)

In [0]:
# 1. Registrar el DataFrame de vendedores como una vista temporal
sellers_df.createOrReplaceTempView("sellers_bronze_vw")

print("Vista temporal 'sellers_bronze_vw' creada.")

In [0]:
%sql
-- Crear la tabla de dimensión de vendedores
-- **CORRECCIÓN**: Añadimos la definición de la columna PK como NOT NULL
CREATE OR REPLACE TABLE sesion_5.silver.dim_vendedores (
  seller_id STRING NOT NULL,
  seller_zip_code_prefix STRING,
  seller_city STRING,
  seller_state STRING
)
USING DELTA
COMMENT 'Dimensión de Vendedores, depurada y modelada.';

-- Insertamos los datos
INSERT INTO sesion_5.silver.dim_vendedores
SELECT DISTINCT
  seller_id,
  seller_zip_code_prefix,
  seller_city,
  seller_state
FROM sellers_bronze_vw;

-- Verificamos
SELECT * FROM sesion_5.silver.dim_vendedores LIMIT 5;

### 5.3. Tabla de Hechos de Pedidos (`fact_pedidos`)
Esta es la tabla más importante. La construiremos uniendo los DataFrames de pedidos, items de pedidos y pagos para consolidar todas las métricas y claves foráneas en un solo lugar.

In [0]:
# 1. Registrar todos los DataFrames relacionados con los pedidos como vistas temporales
orders_df.createOrReplaceTempView("orders_bronze_vw")
order_items_df.createOrReplaceTempView("order_items_bronze_vw")
order_payments_df.createOrReplaceTempView("order_payments_bronze_vw")

print("Vistas temporales para pedidos, items y pagos creadas.")

In [0]:
%sql
-- Crear la tabla de hechos
-- **CORRECCIÓN**: Definimos las columnas que serán FK como NOT NULL
CREATE OR REPLACE TABLE sesion_5.silver.fact_pedidos (
  order_id STRING,
  customer_id STRING NOT NULL,
  product_id STRING NOT NULL,
  seller_id STRING NOT NULL,
  fecha_pedido DATE,
  order_status STRING,
  price DOUBLE,
  costo_envio DOUBLE,
  secuencia_pago INT,
  tipo_pago STRING,
  cuotas_pago INT,
  valor_pago DOUBLE
)
USING DELTA
COMMENT 'Tabla de hechos con las métricas de los pedidos.';

-- Insertamos los datos
INSERT INTO sesion_5.silver.fact_pedidos
SELECT
  o.order_id,
  o.customer_id,
  i.product_id,
  i.seller_id,
  TO_DATE(o.order_purchase_timestamp) AS fecha_pedido,
  o.order_status,
  i.price,
  i.freight_value AS costo_envio,
  p.payment_sequential AS secuencia_pago,
  p.payment_type AS tipo_pago,
  p.payment_installments AS cuotas_pago,
  p.payment_value AS valor_pago
FROM orders_bronze_vw o
JOIN order_items_bronze_vw i ON o.order_id = i.order_id
JOIN order_payments_bronze_vw p ON o.order_id = p.order_id;

-- Verificamos
SELECT * FROM sesion_5.silver.fact_pedidos LIMIT 10;

## Fase 6: Definición de Claves Primarias y Foráneas

**Objetivo:** Establecer formalmente la integridad y las relaciones de nuestro modelo.

Ahora que las tablas existen, vamos a añadirles "restricciones" (constraints).
* **Llaves Primarias (PK)**: Garantizan que cada fila en una tabla de dimensión sea única.
* **Llaves Foráneas (FK)**: Crean un vínculo entre la tabla de hechos y las tablas de dimensión, asegurando que un pedido solo pueda referenciar a clientes y productos que realmente existen.

En el Lakehouse, estas restricciones son informacionales (`NOT ENFORCED`), pero son vitales para que las herramientas de BI, los optimizadores de consultas y los propios desarrolladores entiendan cómo se relacionan las tablas.

In [0]:
%sql
-- Definiendo las Llaves Primarias para cada tabla de dimensión
-- (Usamos la tabla creada con SQL como principal para el resto del taller)
ALTER TABLE sesion_5.silver.dim_clientes_sql ADD CONSTRAINT pk_dim_clientes PRIMARY KEY(customer_id) NOT ENFORCED;
ALTER TABLE sesion_5.silver.dim_productos ADD CONSTRAINT pk_dim_productos PRIMARY KEY(product_id) NOT ENFORCED;
ALTER TABLE sesion_5.silver.dim_vendedores ADD CONSTRAINT pk_dim_vendedores PRIMARY KEY(seller_id) NOT ENFORCED;

-- Definiendo las Llaves Foráneas en la tabla de hechos
ALTER TABLE sesion_5.silver.fact_pedidos ADD CONSTRAINT fk_pedidos_clientes FOREIGN KEY(customer_id) REFERENCES sesion_5.silver.dim_clientes_sql(customer_id) NOT ENFORCED;
ALTER TABLE sesion_5.silver.fact_pedidos ADD CONSTRAINT fk_pedidos_productos FOREIGN KEY(product_id) REFERENCES sesion_5.silver.dim_productos(product_id) NOT ENFORCED;
ALTER TABLE sesion_5.silver.fact_pedidos ADD CONSTRAINT fk_pedidos_vendedores FOREIGN KEY(seller_id) REFERENCES sesion_5.silver.dim_vendedores(seller_id) NOT ENFORCED;

SELECT "Llaves primarias y foráneas definidas exitosamente." as status;

## Fase 7: Verificación Final del Modelo Silver

**Objetivo:** Comprobar que nuestro modelo en estrella funciona correctamente y que las relaciones están bien definidas.

Ahora que tenemos un modelo de datos íntegro y bien estructurado, podemos responder a preguntas de negocio complejas con consultas simples y eficientes.

In [0]:
%sql
-- Pregunta de negocio: ¿Cuál es el valor total pagado por categoría de producto en el estado de São Paulo (SP)?

SELECT
  dp.product_category,
  SUM(fp.valor_pago) AS valor_total_pagado
FROM sesion_5.silver.fact_pedidos AS fp
JOIN sesion_5.silver.dim_clientes_sql AS dc ON fp.customer_id = dc.customer_id
JOIN sesion_5.silver.dim_productos AS dp ON fp.product_id = dp.product_id
WHERE
  dc.customer_state = 'SP'
GROUP BY
  dp.product_category
ORDER BY
  valor_total_pagado DESC
LIMIT 10;

## La Capa Oro: La "Última Milla" de los Datos 🥇

Si la capa Plata es la "fuente única de la verdad" para toda la empresa, la capa **Oro** es la **"versión de la verdad optimizada para un propósito específico"**. Su objetivo no es la integridad o el modelado general, sino la **velocidad y la facilidad de uso** para un caso de negocio concreto.

Piénsalo como la diferencia entre una cocina de restaurante bien surtida (la capa Plata) y el plato final que se sirve en la mesa (la capa Oro). El plato final ya tiene todos los ingredientes combinados, cocinados y presentados de la forma exacta en que el comensal lo necesita.

***

### ## Diferencias Clave: Plata vs. Oro

| Característica | 🥈 Capa Plata (Fuente de la Verdad) | 🥇 Capa Oro (Consumo) |
| :--- | :--- | :--- |
| **Propósito** | Integración y modelado de datos para toda la empresa. | Responder a preguntas de negocio específicas. |
| **Estructura** | Generalmente modelada (ej. Esquema en Estrella). | Altamente desnormalizada y agregada. |
| **Audiencia** | Ingenieros de datos, científicos de datos. | Analistas de negocio, ejecutivos (vía dashboards). |
| **Alcance** | Empresarial (toda la información de clientes). | Específico del proyecto (ej. "rentabilidad del cliente"). |
| **Optimización** | Para la integridad y la flexibilidad del análisis. | Para la velocidad de las consultas (`queries`). |

***

### ## Ejemplos Prácticos para Nuestro Dataset de Olist

A partir de nuestra capa Silver, podríamos crear varias tablas Oro, cada una diseñada para un propósito diferente.

#### Ejemplo 1: El Dashboard de Ventas (BI)
* **Nombre de la tabla**: `resumen_ventas_mensual`
* **Pregunta de negocio**: "¿Cómo van nuestras ventas mes a mes en cada estado?"
* **Estructura**:
  * `año_mes` (ej. '2018-02')
  * `estado_cliente`
  * `total_pedidos` (COUNT)
  * `valor_total_vendido` (SUM)
  * `promedio_por_pedido` (AVG)
  * `total_clientes_unicos` (COUNT DISTINCT)
* **Beneficio**: Una herramienta como Power BI o Tableau puede conectarse a esta tabla y crear un dashboard increíblemente rápido, ya que todos los cálculos pesados ya están hechos. El analista solo arrastra y suelta los campos.

#### Ejemplo 2: El Modelo de Segmentación de Clientes (Machine Learning)
* **Nombre de la tabla**: `features_clientes_rfm`
* **Pregunta de negocio**: "¿Cuáles son nuestros mejores clientes basados en su comportamiento de compra?"
* **Estructura (RFM - Recencia, Frecuencia, Valor Monetario)**:
  * `customer_unique_id`
  * `dias_desde_ultima_compra` (Recencia)
  * `numero_total_pedidos` (Frecuencia)
  * `valor_total_gastado` (Valor Monetario)
  * `categoria_favorita`
* **Beneficio**: Un científico de datos puede usar esta tabla directamente para entrenar un modelo de clustering (ej. K-Means) para segmentar a los clientes en grupos como "Campeones", "En Riesgo", "Nuevos", etc. No necesita hacer `JOINs` ni agregaciones.

#### Ejemplo 3: El Reporte Financiero
* **Nombre de la tabla**: `performance_vendedores`
* **Pregunta de negocio**: "¿Qué vendedores están generando más ingresos y cuál es su costo de envío promedio?"
* **Estructura**:
  * `seller_id`
  * `seller_city`
  * `seller_state`
  * `total_ingresos_generados`
  * `promedio_costo_envio`
  * `numero_productos_vendidos`
* **Beneficio**: El equipo de finanzas o de operaciones puede evaluar rápidamente el rendimiento de los vendedores sin necesidad de consultar múltiples tablas.

***

### ## ¿Por Qué es Tan Importante la Capa Oro?

1.  **Rendimiento Extremo**: Las consultas a tablas agregadas son órdenes de magnitud más rápidas que las consultas que deben procesar millones de filas en la capa Silver. Para un dashboard interactivo, esta es la diferencia entre una experiencia fluida y una frustrante.
2.  **Reducción de Costos**: En sistemas de `data warehousing` en la nube que cobran por datos escaneados, consultar tablas Oro pequeñas y agregadas es mucho más barato que escanear tablas de hechos masivas repetidamente.
3.  **Consistencia del Negocio**: Si todos los reportes de ventas se basan en la tabla `resumen_ventas_mensual`, te aseguras de que todos en la empresa vean exactamente los mismos números. Evita que cada analista calcule las "ventas totales" a su manera.
4.  **Autoservicio Real**: Empodera a los usuarios de negocio para que puedan responder a sus propias preguntas sin depender constantemente del equipo de datos para construir consultas complejas.

## Fase 8: Construcción de la Capa Oro (Gold)

**Objetivo:** Crear tablas agregadas y específicas para el negocio que potencien el BI y el reporting.

La capa Oro es la última milla de nuestro pipeline. No contiene datos nuevos, sino **agregaciones de la capa Silver** diseñadas para ser extremadamente rápidas y fáciles de consultar.

Crearemos una tabla llamada `ventas_por_estado_categoria` que pre-calculará las ventas totales. Un analista de negocio podría conectar una herramienta como Power BI directamente a esta tabla sin necesidad de escribir `JOINs` complejos.

In [0]:
%sql
-- Creamos nuestra primera tabla en la capa Gold
CREATE OR REPLACE TABLE sesion_5.gold.ventas_por_estado_categoria
USING DELTA
COMMENT 'Tabla agregada con el total de ventas por estado del cliente y categoría del producto.'
AS
SELECT
  dc.customer_state,
  dp.product_category,
  COUNT(fp.order_id) AS total_pedidos,
  SUM(fp.valor_pago) AS valor_total_pagado,
  AVG(fp.valor_pago) AS promedio_pago_por_pedido
FROM sesion_5.silver.fact_pedidos AS fp
JOIN sesion_5.silver.dim_clientes_sql AS dc ON fp.customer_id = dc.customer_id
JOIN sesion_5.silver.dim_productos AS dp ON fp.product_id = dp.product_id
GROUP BY
  dc.customer_state,
  dp.product_category;

-- ¡Ahora consultar esta información es instantáneo!
SELECT *
FROM sesion_5.gold.ventas_por_estado_categoria
WHERE customer_state = 'SP'
ORDER BY valor_total_pagado DESC
LIMIT 10;

## ¡Hemos Completado el Pipeline Medallion! 🎉

**¡Felicidades!** Has construido un pipeline de datos de extremo a extremo, desde la ingesta de archivos crudos (Bronze), pasando por la limpieza y modelado (Silver), hasta la creación de una tabla agregada lista para el negocio (Gold).

### Próximos Pasos y Consideraciones Finales

En un proyecto real, los siguientes pasos serían:
* **Optimización y Mantenimiento**: Para asegurar que las consultas a la capa Silver sigan siendo rápidas a medida que los datos crecen, programaríamos tareas de mantenimiento como `OPTIMIZE` y `ZORDER`. Por ejemplo:
    ```sql
    OPTIMIZE sesion_5.silver.fact_pedidos ZORDER BY (fecha_pedido);
    ```
* **Orquestación**: Automatizaríamos la ejecución de este notebook para que se actualice diariamente usando una herramienta como Databricks Workflows o Airflow.
* **Data Quality**: Añadiríamos pruebas de calidad de datos (usando `EXPECTATIONS` en Delta Live Tables, por ejemplo) para asegurar que solo datos válidos pasen de Bronze a Silver.

¡Excelente! Ahora que tienes el pipeline completo, el siguiente paso lógico es automatizarlo. La **orquestación** es el proceso de programar y gestionar la ejecución de tus flujos de trabajo de datos (tus notebooks) de manera automática, fiable y monitorizada.

La herramienta nativa para esto en Databricks se llama **Databricks Workflows (Jobs)**. Es extremadamente potente y fácil de usar.

Aquí tienes un paso a paso detallado para orquestar tu pipeline.

---
## Prerrequisito: Preparar tu Notebook para Producción

Antes de automatizar, es una buena práctica "limpiar" tu notebook de desarrollo para crear una versión de producción. Un notebook de producción no debería tener celdas de exploración o pruebas.

**Acciones recomendadas:**
1.  **Guarda una copia:** Guarda tu notebook actual con un nuevo nombre, por ejemplo, `pipeline_olist_produccion`.
2.  **Elimina celdas innecesarias:** En la nueva copia, elimina las celdas que no son esenciales para la ejecución del pipeline:
    * Las celdas de exploración (EDA) con `.show()` o `.printSchema()`.
    * La celda de comparación entre PySpark y SQL. Quédate solo con la versión que usarás en producción (la versión SQL en nuestro caso).
    * La celda de `Fase 0` para crear el catálogo y los esquemas. En producción, la infraestructura ya debería existir.
3.  **Asegúrate de que se ejecute de principio a fin:** El notebook final debe contener únicamente el código necesario para ejecutar la transformación de Bronze a Gold sin errores.

---
## Paso a Paso para la Orquestación con Databricks Workflows

### Paso 1: Ir a la Sección de Workflows

En el menú principal de la izquierda de tu workspace de Databricks, haz clic en el ícono de **Workflows**.


---
### Paso 2: Crear un Nuevo "Job"

Un **Job** (Trabajo) es el contenedor de tu flujo de trabajo. Puede tener una o varias tareas.

1.  Dentro de la página de Workflows, haz clic en el botón azul **Create Job**.
2.  Dale un nombre descriptivo a tu Job en la parte superior, por ejemplo, `Procesamiento Diario Olist`.



---
### Paso 3: Configurar tu Primera Tarea (Task)

Ahora, configurarás la tarea que ejecutará tu notebook.

1.  **Nombre de la Tarea**: Dale un nombre a la tarea, como `bronze_a_gold`.
2.  **Tipo (Type)**: Selecciona **Notebook**.
3.  **Fuente (Source)**: Selecciona **Workspace** y busca tu notebook de producción (`pipeline_olist_produccion.ipynb`) usando el explorador de archivos.
4.  **Cluster**: Aquí tienes la decisión más importante.
    * **Opción A (Recomendada para producción):** *New Job Cluster*. Esto crea un clúster temporal que se enciende solo para ejecutar el job y se apaga al terminar. Es la opción más eficiente en costos. Puedes configurar un tamaño de clúster pequeño para empezar.
    * **Opción B (Para pruebas):** Puedes seleccionar tu clúster interactivo existente (el que usaste para desarrollar).
5.  Haz clic en el botón **Create task**.



---
### Paso 4: Programar la Ejecución (Schedule)

Aquí es donde le dices a Databricks *cuándo* quieres que se ejecute tu pipeline.

1.  En el panel de la derecha de la configuración del Job, haz clic en **Add trigger**.
2.  En "Trigger type", selecciona **Schedule**.
3.  Configura la frecuencia. Por ejemplo, para una ejecución diaria a las 3:00 AM:
    * **Schedule**: `Daily`
    * **At**: `03` : `00`
4.  Puedes usar la sintaxis **Cron** para programaciones más complejas si lo necesitas.
5.  Haz clic en **Save**.



---
### Paso 5: (Opcional pero recomendado) Configurar Alertas

Es vital saber si tu pipeline falló o se completó con éxito.

1.  En la configuración del Job, ve a la sección de **Notifications**.
2.  Haz clic en **Add notification**.
3.  Añade tu correo electrónico para los eventos **On success** (éxito) y, más importante, **On failure** (fallo).
4.  Haz clic en **Confirm**.

---
### Paso 6: Ejecutar y Monitorear

¡Listo! Ya tienes un pipeline orquestado.

* Puedes hacer clic en **Run now** en la esquina superior derecha para probarlo inmediatamente.
* En la pestaña **Runs**, podrás ver el historial de todas las ejecuciones, ver si tuvieron éxito o fallaron, y acceder a los logs para depurar cualquier problema.

Siguiendo estos pasos, has pasado de un script interactivo a un pipeline de datos automatizado, robusto y listo para producción.

## Orquestación del Pipeline con Databricks Workflows ⚙️

**Objetivo:** Automatizar la ejecución de nuestro pipeline para que se actualice de forma regular sin intervención manual.

Un pipeline solo es útil si los datos están frescos. La orquestación es el proceso de programar nuestro notebook para que se ejecute automáticamente. La herramienta nativa para esto es **Databricks Workflows (Jobs)**.

**Pasos para Orquestar este Notebook:**
1.  **Preparar una Versión de Producción**: Guarda una copia de este notebook (ej. `pipeline_olist_produccion`) y elimina las celdas de exploración (EDA) y las de comparación (PySpark vs SQL). El notebook final debe contener solo el código esencial para ir de Bronce a Oro.
2.  **Ir a Workflows**: En el menú de Databricks, ve a la sección "Workflows".
3.  **Crear un Nuevo Job**: Dale un nombre (ej. `Procesamiento Diario Olist`).
4.  **Configurar la Tarea**:
    * **Tipo**: Notebook.
    * **Fuente**: Selecciona tu notebook de producción.
    * **Cluster**: Configura un *New Job Cluster*. Esto es más eficiente en costos, ya que el clúster se crea solo para la ejecución y se apaga al terminar.
5.  **Programar la Ejecución (Schedule)**:
    * Añade un "Trigger" de tipo "Schedule".
    * Configúralo para que se ejecute diariamente a la hora que prefieras (ej. 3:00 AM).
6.  **Configurar Alertas**: Añade tu correo electrónico en la sección de "Notifications" para recibir una alerta si el job falla.

Una vez configurado, Databricks se encargará de ejecutar tu pipeline y mantener tus datos actualizados automáticamente.

## Mantenimiento y Optimización a Largo Plazo 🛠️

**Objetivo:** Entender las tareas necesarias para mantener el rendimiento de nuestro Lakehouse.

A medida que los datos crecen, las consultas pueden volverse más lentas. Delta Lake viene con herramientas para combatir esto. En un entorno de producción, añadiríamos estos comandos al final de nuestro job orquestado:

* **`OPTIMIZE`**: Compacta los archivos pequeños en archivos más grandes. Leer un archivo grande es mucho más rápido para Spark que leer miles de archivos pequeños.
* **`ZORDER`**: Es una técnica de co-localización de datos. Ordena los datos dentro de los archivos según las columnas que más usas para filtrar (como fechas o IDs), permitiendo a Spark "saltarse" los archivos que no necesita leer.

In [0]:
%sql
-- Ejemplo de comandos de mantenimiento que se ejecutarían al final del pipeline
-- Esto no necesita ejecutarse ahora, es para ilustrar el concepto.

-- OPTIMIZE sesion_5.silver.fact_pedidos ZORDER BY (fecha_pedido);
-- ANALYZE TABLE sesion_5.silver.fact_pedidos COMPUTE STATISTICS FOR ALL COLUMNS;

SELECT "El mantenimiento es clave para un rendimiento sostenido." as conclusion;

## Conclusión de la Reconstrucción Punta a Punta 🚀

**¡Lo hemos logrado!** Hemos construido un pipeline de datos completo, robusto y automatizable, siguiendo las mejores prácticas de la industria.

**Nuestro Viaje:**
1.  **Configuramos una Arquitectura Profesional**: Creamos un catálogo dedicado con esquemas y volúmenes para cada capa (Bronce, Plata, Oro).
2.  **Ingestamos Datos Crudos (Bronce)**: Cargamos los archivos CSV originales sin ninguna modificación.
3.  **Validamos la Calidad**: Implementamos "guardianes" que detienen el pipeline si los datos no cumplen con nuestras reglas de negocio, protegiendo la integridad de nuestro Lakehouse.
4.  **Modelamos la Fuente de la Verdad (Plata)**: Transformamos los datos crudos en un Esquema en Estrella limpio, validado y con relaciones definidas, listo para análisis flexibles.
5.  **Entregamos Valor al Negocio (Oro)**: Creamos tablas pre-agregadas y optimizadas para que las consultas de BI sean instantáneas.

Ahora tienes un modelo de trabajo que puedes adaptar y escalar para cualquier proyecto de datos en el futuro.